In [1]:
import numpy as np
import pandas as pd
import os
import json
import sys

In [2]:
data_dir = "/Users/roopal/workspace/kaggle/quora_question_pairs/data"

In [3]:
train_file = os.path.join(data_dir, "train.csv")
test_file = os.path.join(data_dir, "test.csv")
sample_submission = os.path.join(data_dir, "sample_submission.csv")

In [4]:
df_train = pd.read_csv(train_file, index_col=False)

In [5]:
df_train = df_train.dropna()

In [6]:
REPO_LOCAL_PATH = "/Users/roopal/ggrepo/ds-tws-backend"

In [7]:
if REPO_LOCAL_PATH not in sys.path:
    sys.path.append(REPO_LOCAL_PATH)
    print "repo local path added"
else:
    print "repo local path already added"

repo local path added


In [8]:
from util.util_text_process import get_sentence_tokenizer, get_sentences, tokenize as tokenize_en
from deep_learning.embeddings.Word2VecEmbeddings import Word2VecEmbeddings
from deep_learning.corpus_reader.CorpusReader import CorpusReader

In [9]:
wrd2idx, _ = Word2VecEmbeddings.get_embeddings_with_custom_tokens(
    '/Users/roopal/workspace/kaggle/quora_question_pairs/data/w2v.model.bin', binary=True
)

75517
126790
93153
128059
33072
123344
110474
106841
109880
77163
99722
121716
71599
100680
73005
105561
73425
100906
25057
115515
116461
114756
61857
46360
115287
123233
53784
126835
122720
74398
79246
31419
122737
31628
27050
113791
104262
101729
95669
99079
96218
125787
102513
43731
97887
85086
59179
117986


In [23]:
WORD2IDX = wrd2idx

In [24]:
EN_SENT_TOKENIZER = get_sentence_tokenizer()

In [25]:
def process_sentences(sentences, word2idx, build_word2idx=False):
    list_line_word_idx = list()
    for sentence in sentences:
        sentence = CorpusReader.clean_line(sentence)
        
        words = tokenize_en(sentence)
        
        line_word_idx = list()
        for word in words:
            idx = CorpusReader.get_word_index(word, word2idx, build_word2idx)
            line_word_idx.append(idx)
        list_line_word_idx.append(line_word_idx)
        
    return list_line_word_idx

In [26]:
def get_question_sent_idx(question1):
    try:
        q1 = process_sentences([question1], WORD2IDX)
    except Exception as e:
        print str(e), question1
        q1 = [[]]
        
#     try:
#         q2 = process_sentences([question2], WORD2IDX)
#     except Exception as e:
#         print str(e), question2
#         q2 = []
#     return id, q1, q2
    return q1

In [27]:
df_train["q1_sent_idx"] = df_train["question1"].apply(lambda x: get_question_sent_idx(x))

In [28]:
df_train["q2_sent_idx"] = df_train["question2"].apply(lambda x: get_question_sent_idx(x))

In [29]:
df_train.groupby(["is_duplicate"]).agg(["count"])

,id,qid1,qid2,question1,question2,q1_sent_idx,q2_sent_idx
,count,count,count,count,count,count,count
is_duplicate,,,,,,,
0,255025,255025,255025,255025,255025,255025,255025
1,149263,149263,149263,149263,149263,149263,149263


In [30]:
df_train.to_pickle(train_file+".q_w2v.pkl")

In [31]:
df_test = pd.read_csv(test_file, index_col=False)

In [32]:
df_test["q1_sent_idx"] = df_test["question1"].apply(lambda x: get_question_sent_idx(x))

expected string or buffer nan
expected string or buffer nan


In [33]:
df_test["q2_sent_idx"] = df_test["question2"].apply(lambda x: get_question_sent_idx(x))

expected string or buffer nan
expected string or buffer nan
expected string or buffer nan
expected string or buffer nan


In [34]:
df_test.to_pickle(test_file+".q_w2v.pkl")

In [35]:
for item, frame in df_test["q1_sent_idx"].iteritems():
    if len(frame) == 1 and len(frame[0]) > 0:
        continue
    print item, frame

94646 [[]]
714289 [[]]
805015 [[]]
913286 [[]]
923320 [[]]
1046690 [[]]
1221596 [[]]
1461432 [[]]


In [36]:
for item, frame in df_test["q2_sent_idx"].iteritems():
    if len(frame) == 1 and len(frame[0]) > 0:
        continue
    print item, frame

379205 [[]]
788917 [[]]
817520 [[]]
856433 [[]]
943911 [[]]
963864 [[]]
1250219 [[]]
1270024 [[]]
1635393 [[]]
2164990 [[]]


In [37]:
print "done..."

done...
